In [ ]:
## Module import
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from scipy import stats
import seaborn as sns
import cartopy.feature as cft
import matplotlib.path as mpath
import cmocean as cm
import string
import math
from datetime import datetime, timedelta
import proplot as pplt
import xesmf as xe
import netCDF4 as nc

## CMIP6 Output

In [ ]:
# open IPSL file
netCDF_T95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/data_T_for_std_19952014.nc'
TEMP_CMIP95 = xr.open_mfdataset(netCDF_T95_file)

netCDF_S95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/data_S_for_std_19952014.nc'
SAL_CMIP95 = xr.open_mfdataset(netCDF_S95_file)

netCDF_SI95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/data_SI_for_std_19952014.nc'
SI_CMIP95 = xr.open_mfdataset(netCDF_SI95_file)

# open MPI file
netCDF_MPI_T95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/data_MPI_T_for_std_19952014.nc'
TEMP_MPI_CMIP95 = xr.open_mfdataset(netCDF_MPI_T95_file)

netCDF_MPI_S95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/data_MPI_S_for_std_19952014.nc'
SAL_MPI_CMIP95 = xr.open_mfdataset(netCDF_MPI_S95_file)

netCDF_MPI_SI95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/data_MPI_SI_for_std_19952014.nc'
SI_MPI_CMIP95 = xr.open_mfdataset(netCDF_MPI_SI95_file)

# open UKESM file
netCDF_UKESM_T95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/data_UKESM_T_for_std_19952014.nc'
TEMP_UKESM_CMIP95 = xr.open_mfdataset(netCDF_UKESM_T95_file)

netCDF_UKESM_S95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/data_UKESM_S_for_std_19952014.nc'
SAL_UKESM_CMIP95 = xr.open_mfdataset(netCDF_UKESM_S95_file)

netCDF_UKESM_SI95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/data_UKESM_SI_for_std_19952014.nc'
SI_UKESM_CMIP95 = xr.open_mfdataset(netCDF_UKESM_SI95_file)

# open topo
netCDF_topom_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/deptho_Ofx_MPI-ESM1-2-HR_piControl_r1i1p1f1_gn.nc'
TOPOM = xr.open_mfdataset(netCDF_topom_file)

netCDF_topou_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/deptho_Ofx_UKESM1-0-LL_piControl_r1i1p1f2_gn.nc'
TOPOU = xr.open_mfdataset(netCDF_topou_file)

#### correction fichier topo pour contour line

In [ ]:
delta_lon = np.abs(np.diff(TOPOM['longitude']))
for i, start in enumerate(np.argmax(delta_lon > 180, axis=1)):
    TOPOM['longitude'][i, start+1:] += 360
    
delta_lon1 = np.abs(np.diff(TOPOU['longitude']))
for i, start in enumerate(np.argmax(delta_lon1 > 180, axis=1)):
    TOPOU['longitude'][i, start+1:] += 360

## Observations

In [ ]:
### OBS
# Temperature 1995-2017
netCDF_fileT0 = '/Users/jcaillet/Documents/Elmer/CMIP6/OBSERVATIONS/T/woa18_1995-2004_t00_01.nc'
netCDF_fileT1 = '/Users/jcaillet/Documents/Elmer/CMIP6/OBSERVATIONS/T/woa18_2005-2017_t00_01.nc'
OBST095 = xr.open_mfdataset(netCDF_fileT0, decode_cf=False)
OBST195 = xr.open_mfdataset(netCDF_fileT1, decode_cf=False)
OBST95 = xr.merge([OBST095,OBST195])
OBST95 = OBST95.mean('time')
OBST95['temp']= xr.where(OBST95.t_an>1000,np.nan,OBST95.t_an.values)
OBST95_sel = OBST95.sel(depth=slice(200.0, 700.0)).mean('depth')

# Salinity 1995-2017
netCDF_fileS0 = '/Users/jcaillet/Documents/Elmer/CMIP6/OBSERVATIONS/S/woa18_1995-2004_s00_01.nc'
netCDF_fileS1 = '/Users/jcaillet/Documents/Elmer/CMIP6/OBSERVATIONS/S/woa18_2005-2017_s00_01.nc'
OBSS095 = xr.open_mfdataset(netCDF_fileS0, decode_cf=False)
OBSS195 = xr.open_mfdataset(netCDF_fileS1, decode_cf=False)
OBSS95 = xr.merge([OBSS095,OBSS195])
OBSS95 = OBSS95.mean('time')
OBSS95['sal'] = xr.where(OBSS95.s_an>1000,np.nan,OBSS95.s_an.values)
OBSS95_sel = OBSS95.sel(depth=slice(200.0, 700.0)).mean('depth')


## preliminary step : open observation file version 3 for adding latitude, longitude in version 4
netCDF_siconc_obs_file0 = '/Users/jcaillet/Documents/nemo/DATA_OBSERVATIONS/Sea_ice_concentration/seaice_conc_monthly_sh_n07_197910_v03r01.nc'
SICONC_OBS0 = xr.open_mfdataset(netCDF_siconc_obs_file0)
SICONC_OBS0 = SICONC_OBS0.rename_dims({'xgrid':'x', 'ygrid':'y'})

## open sea-ice observation files - version 4
netCDF_siconc_obs_file = '/Users/jcaillet/Documents/Elmer/CMIP6/OBSERVATIONS/Sea_ice/*.nc'
SICONC_OBS = xr.open_mfdataset(netCDF_siconc_obs_file, concat_dim='tdim', combine='nested')
# add longitude - latitude
SICONC_OBS['longitude'] = SICONC_OBS0['longitude']
SICONC_OBS = SICONC_OBS.set_coords(("longitude"))
SICONC_OBS['year'] = SICONC_OBS['time'].dt.year
SICONC_OBSy = SICONC_OBS.groupby('year').mean()
SICONC_OBSy_sel = SICONC_OBSy.sel(year=slice('1995', '2014')).mean('year')
SICONC_OBSy_sel['conc'] = xr.where(SICONC_OBSy_sel.cdr_seaice_conc_monthly>1,np.nan,SICONC_OBSy_sel.cdr_seaice_conc_monthly.values)

## FIGURE

In [ ]:
sns.set_context('paper')

land_50m = cft.NaturalEarthFeature('physical', 'land', '50m', edgecolor='salmon', facecolor='papayawhip', linewidth=0.5)
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

fig, axs = plt.subplots(nrows = 3, ncols = 4, subplot_kw={'projection': ccrs.SouthPolarStereo()}, figsize = (11.5, 8), constrained_layout=True)
axs = axs.flatten()


for n, ax in enumerate(axs):
    ax.text(0.025, 0.93, string.ascii_lowercase[n]+')', transform=ax.transAxes,color='black', fontweight="bold")
    ax.set_extent([-180, 180, -90, -63], crs=ccrs.PlateCarree())
    ax.add_feature(land_50m, color=[0.8, 0.8, 0.8])
    ax.coastlines(resolution='50m', color='black',)
    gl0 = ax.gridlines(draw_labels=False, linewidth=0.75, color='gray', alpha=0.7, linestyle='--', dms=True, x_inline=False, y_inline=False)
    gl0.top_labels = False
    gl0.right_labels = False
    gl0.xlocator = mticker.FixedLocator([-180, -135, -90, -45, 0, 45, 90, 135, 180])
    gl0.ylocator = mticker.FixedLocator([-90, -80, -70,-60])
    gl0.xformatter = LongitudeFormatter()
    gl0.yformatter = LatitudeFormatter()
    gl0.xlabel_style = {'size': 12, 'color': 'gray'}
    gl0.ylabel_style = {'size': 12, 'color': 'gray'}
    #ax.set_boundary(circle, transform=ax.transAxes)
    
plt.figtext(0.01, 0.8, '1995-2014 multi-member mean', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.025, 0.8, 'of practical salinity', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.01, 0.485, '1995-2014 multi-member mean', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.025, 0.485, 'of potential temperature', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.422, 0.97, 'MPI', fontweight="bold")
plt.figtext(0.646, 0.97, 'IPSL', fontweight="bold")
plt.figtext(0.86, 0.97, 'UKESM', fontweight="bold")
plt.figtext(0.165, 0.97, 'OBSERVATIONS', fontweight="bold")
plt.figtext(0.01, 0.165, '1995-2014 multi-member mean', horizontalalignment='left', verticalalignment='center', rotation='vertical', fontweight="bold")
plt.figtext(0.025, 0.165, 'of sea-ice concentration', horizontalalignment='left', verticalalignment='center', rotation='vertical', fontweight="bold")


# oceanic temperature 1995-2014
# Salinity 1995-2014
p3 = axs[6].pcolormesh(TEMP_CMIP95.nav_lon, TEMP_CMIP95.nav_lat, TEMP_CMIP95['Temp_mean_200700'],  cmap=cm.cm.thermal, vmin=-2, vmax=3, transform=ccrs.PlateCarree(), shading='auto')
p33 = axs[6].contour(TOPOU.longitude, TOPOU.latitude, TOPOU['deptho'], levels=[1500], colors='lightskyblue', linewidths = 1, transform=ccrs.PlateCarree())

p4 = axs[5].pcolormesh(TEMP_MPI_CMIP95.longitude, TEMP_MPI_CMIP95.latitude, TEMP_MPI_CMIP95['Temp_mean_200700'],  cmap=cm.cm.thermal,  vmin=-2, vmax=3, transform=ccrs.PlateCarree(), shading='auto')
p44 = axs[5].contour(TOPOM.longitude, TOPOM.latitude, TOPOM['deptho'], levels=[1500], colors='lightskyblue', linewidths = 1, transform=ccrs.PlateCarree())

p04 = axs[4].pcolormesh(OBST95_sel.lon, OBST95_sel.lat, OBST95_sel['temp'],  cmap=cm.cm.thermal,  vmin=-2, vmax=3, transform=ccrs.PlateCarree(), shading='auto')

p5 = axs[7].pcolormesh(TEMP_UKESM_CMIP95.longitude, TEMP_UKESM_CMIP95.latitude, TEMP_UKESM_CMIP95['Temp_mean_200700'],  cmap=cm.cm.thermal,  vmin=-2, vmax=3, transform=ccrs.PlateCarree(), shading='auto')
p55 = axs[7].contour(TOPOU.longitude, TOPOU.latitude, TOPOU['deptho'], levels=[1500], colors='lightskyblue', linewidths = 1, transform=ccrs.PlateCarree())

# oceanic salinity 1995-2014
p0 = axs[2].pcolormesh(SAL_CMIP95.nav_lon, SAL_CMIP95.nav_lat, SAL_CMIP95['Sal_mean_200700'], cmap=cm.cm.haline,  vmin=34, vmax=34.8, transform=ccrs.PlateCarree(), shading='auto')
p00 = axs[2].contour(TOPOU.longitude, TOPOU.latitude, TOPOU['deptho'], levels=[1500], colors='salmon', linewidths = 1, transform=ccrs.PlateCarree())

p1 = axs[1].pcolormesh(SAL_MPI_CMIP95.longitude, SAL_MPI_CMIP95.latitude, SAL_MPI_CMIP95['Sal_mean_200700'], cmap=cm.cm.haline, vmin=34, vmax=34.8, transform=ccrs.PlateCarree(), shading='auto')
p11 = axs[1].contour(TOPOM.longitude, TOPOM.latitude, TOPOM['deptho'], levels=[1500], colors='salmon', linewidths = 1,  transform=ccrs.PlateCarree())

p01 = axs[0].pcolormesh(OBSS95_sel.lon, OBSS95_sel.lat, OBSS95_sel['sal'], cmap=cm.cm.haline, vmin=34, vmax=34.8, transform=ccrs.PlateCarree(), shading='auto')

p2 = axs[3].pcolormesh(SAL_UKESM_CMIP95.longitude, SAL_UKESM_CMIP95.latitude, SAL_UKESM_CMIP95['Sal_mean_200700'], cmap=cm.cm.haline, vmin=34, vmax=34.8, transform=ccrs.PlateCarree(), shading='auto')
p22 = axs[3].contour(TOPOU.longitude, TOPOU.latitude, TOPOU['deptho'], levels=[1500], colors='salmon', linewidths = 1, transform=ccrs.PlateCarree())


# smb 1995-2014
p6 = axs[10].pcolormesh(SI_CMIP95.nav_lon, SI_CMIP95.nav_lat, SI_CMIP95['SI_mean_200700'], cmap=cm.cm.dense, vmin = 0, vmax = 1, transform=ccrs.PlateCarree(), shading='auto')
p66 = axs[10].contour(TOPOU.longitude, TOPOU.latitude, TOPOU['deptho'], levels=[1500], colors='salmon', linewidths = 1, transform=ccrs.PlateCarree())

p7 = axs[9].pcolormesh(SI_MPI_CMIP95.longitude, SI_MPI_CMIP95.latitude, SI_MPI_CMIP95['SI_mean_200700'], cmap=cm.cm.dense, vmin = 0, vmax = 1,  transform=ccrs.PlateCarree(), shading='auto')
p77 = axs[9].contour(TOPOM.longitude, TOPOM.latitude, TOPOM['deptho'], levels=[1500], colors='salmon', linewidths = 1, transform=ccrs.PlateCarree())

p07 = axs[8].pcolormesh(SICONC_OBSy_sel.longitude, SICONC_OBSy_sel.latitude, SICONC_OBSy_sel['conc'], cmap=cm.cm.dense, vmin = 0, vmax = 1, transform=ccrs.PlateCarree(), shading='auto')

p8 = axs[11].pcolormesh(SI_UKESM_CMIP95.longitude, SI_UKESM_CMIP95.latitude, SI_UKESM_CMIP95['SI_mean_200700'], cmap=cm.cm.dense, vmin = 0, vmax = 1,  transform=ccrs.PlateCarree(), shading='auto')
p88 = axs[11].contour(TOPOU.longitude, TOPOU.latitude, TOPOU['deptho'], levels=[1500], colors='salmon', linewidths = 1, transform=ccrs.PlateCarree())

ax_cb = plt.axes([0.047, 0.675, 0.007, 0.25 ])
cb = plt.colorbar(p01, cax=ax_cb, orientation='vertical')
cb.ax.set_ylabel('Practical Salinity (g kg$^{-1}$)');

ax_cb1 = plt.axes([0.047, 0.36, 0.007, 0.25 ])
cb1 = plt.colorbar(p04, cax=ax_cb1, orientation='vertical')
cb1.ax.set_ylabel('Potential Temperature (°C)');

ax_cb2 = plt.axes([0.047, 0.04, 0.007, 0.25 ])
cb2 = plt.colorbar(p07, cax=ax_cb2, orientation='vertical')
cb2.ax.set_ylabel('Sea-ice Concentration (-)');

axs[0].text(-56, -64.6, 'Weddell Sea', color='black', fontsize=8, transform=ccrs.PlateCarree(), zorder=50)
axs[0].text(210, -72.5, 'Ross Sea', color='black', fontsize=8, transform=ccrs.PlateCarree(), zorder=50)
axs[0].text(-112, -61.6, 'Amundsen \n Sea', color='black', fontsize=8, transform=ccrs.PlateCarree(), zorder=50)
axs[0].text(78, -68.8, 'Prydz \n Bay', color='black', fontsize=8, transform=ccrs.PlateCarree(), zorder=50)

plt.tight_layout(rect=[0.095, 0, 1, 0.97])
plt.savefig('/Users/jcaillet/Documents/Elmer/CMIP6/FIGURES/Figure4_mean19952014_modif.png',dpi=190)